# Import Libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.pyplot as plotter
import re
import warnings
warnings.filterwarnings('ignore')


# Import Datasets

In [2]:
train=pd.read_excel("C:/Users/trinity/Downloads/BooStudioLLC.8ZipLite_b6e429xa66pga!App/Extracted/Restaurant_food_cost/Participants_Data_Final/Data_Train.xlsx")
test=pd.read_excel("C:/Users/trinity/Downloads/BooStudioLLC.8ZipLite_b6e429xa66pga!App/Extracted/Restaurant_food_cost/Participants_Data_Final/Data_Test.xlsx")

In [3]:
train.head()

,TITLE,RESTAURANT_ID,CUISINES,TIME,CITY,LOCALITY,RATING,VOTES,COST
0,CASUAL DINING,9438,"Malwani, Goan, North Indian","11am – 4pm, 7:30pm – 11:30pm (Mon-Sun)",Thane,Dombivali East,3.6,49 votes,1200
1,"CASUAL DINING,BAR",13198,"Asian, Modern Indian, Japanese",6pm – 11pm (Mon-Sun),Chennai,Ramapuram,4.2,30 votes,1500
2,CASUAL DINING,10915,"North Indian, Chinese, Biryani, Hyderabadi","11am – 3:30pm, 7pm – 11pm (Mon-Sun)",Chennai,Saligramam,3.8,221 votes,800
3,QUICK BITES,6346,"Tibetan, Chinese",11:30am – 1am (Mon-Sun),Mumbai,Bandra West,4.1,24 votes,800
4,DESSERT PARLOR,15387,Desserts,11am – 1am (Mon-Sun),Mumbai,Lower Parel,3.8,165 votes,300


In [4]:
test.head()

,TITLE,RESTAURANT_ID,CUISINES,TIME,CITY,LOCALITY,RATING,VOTES
0,CASUAL DINING,4085,"North Indian, Chinese, Mughlai, Kebab",12noon – 12midnight (Mon-Sun),Noida,Sector 18,4.3,564 votes
1,QUICK BITES,12680,"South Indian, Fast Food, Pizza, North Indian",7am – 12:30AM (Mon-Sun),Mumbai,Grant Road,4.2,61 votes
2,CASUAL DINING,1411,"North Indian, Seafood, Biryani, Chinese",11am – 11:30pm (Mon-Sun),Mumbai,Marine Lines,3.8,350 votes
3,None,204,Biryani,"9am – 10pm (Mon, Wed, Thu, Fri, Sat, Sun), 10:...",Faridabad,NIT,3.8,1445 votes
4,QUICK BITES,13453,"South Indian, Kerala",11am – 10pm (Mon-Sun),Kochi,Kaloor,3.6,23 votes


In [5]:
# The shape of the dataset
print("food_train",train.shape)
print("food_test",test.shape)

food_train (12690, 9)
food_test (4231, 8)


In [6]:
df = train.append(test,ignore_index=True)

# Data Exploration

In [7]:
train.duplicated().sum(),test.duplicated().sum()

(25, 1)

In [8]:
#check the column names
train.columns

Index(['TITLE', 'RESTAURANT_ID', 'CUISINES', 'TIME', 'CITY', 'LOCALITY',
       'RATING', 'VOTES', 'COST'],
      dtype='object')

In [9]:
test.columns

Index(['TITLE', 'RESTAURANT_ID', 'CUISINES', 'TIME', 'CITY', 'LOCALITY',
       'RATING', 'VOTES'],
      dtype='object')

In [10]:
for i in train.columns:
    print("Unique values in", i,train[i].nunique())

Unique values in TITLE 113
Unique values in RESTAURANT_ID 11892
Unique values in CUISINES 4155
Unique values in TIME 2689
Unique values in CITY 359
Unique values in LOCALITY 1416
Unique values in RATING 32
Unique values in VOTES 1847
Unique values in COST 86


From above observation the data in food train and food test is same

# Data Preprocessing

In [11]:
df.isnull().sum()

TITLE               0
RESTAURANT_ID       0
CUISINES            0
TIME                0
CITY              147
LOCALITY          128
RATING              4
VOTES            1606
COST             4231
dtype: int64

###### Handling missing values

In [12]:
df['RATING'] = df['RATING'].str.replace('NEW', '1')
df['RATING'] = df['RATING'].str.replace('-', '1').astype(float)


In [13]:
df["RATING"].fillna(0.0,inplace=True)

In [14]:
# Wechange VOTES to VOTE and take only numeric value. Drop VOTES column
df['VOTES'] = df['VOTES'].str.replace('votes', '').astype(float)
df["VOTES"].fillna(0.0,inplace=True)

In [15]:
df["CITY"].fillna('NONE',inplace=True)

In [16]:
df["LOCALITY"].fillna('NONE',inplace=True)

In [17]:
df.isnull().sum()

TITLE               0
RESTAURANT_ID       0
CUISINES            0
TIME                0
CITY                0
LOCALITY            0
RATING              0
VOTES               0
COST             4231
dtype: int64

In [18]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16921 entries, 0 to 16920
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   TITLE          16921 non-null  object 
 1   RESTAURANT_ID  16921 non-null  int64  
 2   CUISINES       16921 non-null  object 
 3   TIME           16921 non-null  object 
 4   CITY           16921 non-null  object 
 5   LOCALITY       16921 non-null  object 
 6   RATING         16921 non-null  float64
 7   VOTES          16921 non-null  float64
 8   COST           12690 non-null  float64
dtypes: float64(3), int64(1), object(5)
memory usage: 1.2+ MB


# Summary Statistics

In [19]:
df.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
RESTAURANT_ID,16921.0,7767.211631,4493.287626,0.0,3873.0,7751.0,11672.0,15573.0
RATING,16921.0,3.519957,0.911555,0.0,3.4,3.8,4.0,4.9
VOTES,16921.0,379.407836,808.451565,0.0,40.0,133.0,409.0,41186.0
COST,12690.0,655.252246,627.003540,20.0,300.0,500.0,800.0,14000.0


# Exploratory Analysis

In [20]:
df.head()

,TITLE,RESTAURANT_ID,CUISINES,TIME,CITY,LOCALITY,RATING,VOTES,COST
0,CASUAL DINING,9438,"Malwani, Goan, North Indian","11am – 4pm, 7:30pm – 11:30pm (Mon-Sun)",Thane,Dombivali East,3.6,49.0,1200.0
1,"CASUAL DINING,BAR",13198,"Asian, Modern Indian, Japanese",6pm – 11pm (Mon-Sun),Chennai,Ramapuram,4.2,30.0,1500.0
2,CASUAL DINING,10915,"North Indian, Chinese, Biryani, Hyderabadi","11am – 3:30pm, 7pm – 11pm (Mon-Sun)",Chennai,Saligramam,3.8,221.0,800.0
3,QUICK BITES,6346,"Tibetan, Chinese",11:30am – 1am (Mon-Sun),Mumbai,Bandra West,4.1,24.0,800.0
4,DESSERT PARLOR,15387,Desserts,11am – 1am (Mon-Sun),Mumbai,Lower Parel,3.8,165.0,300.0


In [21]:
df=df[['TITLE', 'CUISINES', 'TIME', 'CITY', 'LOCALITY','RATING','VOTES', 'COST']]
def extract_closed(time):
    a = re.findall('Closed \(.*?\)', time)
    if a != []:
        return a[0]
    else:
        return 'NA'
df['CLOSED'] = train['TIME'].apply(extract_closed)

In [22]:
df['TIME'] = df['TIME'].str.replace(r'Closed \(.*?\)','')

In [23]:
calc= df.groupby(['CITY'], axis=0).agg({'RATING':['mean']}).reset_index()
calc.columns = ['CITY','CITY_MEAN_RATING']
df =df.merge(calc, on=['CITY'],how='left')

calc = df.groupby(['LOCALITY'], axis=0).agg({'RATING':['mean']}).reset_index()
calc.columns = ['LOCALITY','LOCALITY_MEAN_RATING']
df= df.merge(calc, on=['LOCALITY'],how='left')

In [24]:
df.head()

,TITLE,CUISINES,TIME,CITY,LOCALITY,RATING,VOTES,COST,CLOSED,CITY_MEAN_RATING,LOCALITY_MEAN_RATING
0,CASUAL DINING,"Malwani, Goan, North Indian","11am – 4pm, 7:30pm – 11:30pm (Mon-Sun)",Thane,Dombivali East,3.6,49.0,1200.0,NA,3.376271,3.388889
1,"CASUAL DINING,BAR","Asian, Modern Indian, Japanese",6pm – 11pm (Mon-Sun),Chennai,Ramapuram,4.2,30.0,1500.0,NA,3.584588,3.472222
2,CASUAL DINING,"North Indian, Chinese, Biryani, Hyderabadi","11am – 3:30pm, 7pm – 11pm (Mon-Sun)",Chennai,Saligramam,3.8,221.0,800.0,NA,3.584588,3.550000
3,QUICK BITES,"Tibetan, Chinese",11:30am – 1am (Mon-Sun),Mumbai,Bandra West,4.1,24.0,800.0,NA,3.697880,3.721622
4,DESSERT PARLOR,Desserts,11am – 1am (Mon-Sun),Mumbai,Lower Parel,3.8,165.0,300.0,NA,3.697880,3.986420


In [25]:
from sklearn.feature_extraction.text import TfidfVectorizer
tf1 = TfidfVectorizer(ngram_range=(1, 1), lowercase=True)
df_title = tf1.fit_transform(df['TITLE'])
df_title = pd.DataFrame(data=df_title.toarray(), columns=tf1.get_feature_names())

tf2 = TfidfVectorizer(ngram_range=(1, 1), lowercase=True)
df_cuisines = tf2.fit_transform(df['CUISINES'])
df_cuisines = pd.DataFrame(data=df_cuisines.toarray(), columns=tf2.get_feature_names())

tf3 = TfidfVectorizer(ngram_range=(1, 1), lowercase=True)
df_city = tf3.fit_transform(df['CITY'])
df_city = pd.DataFrame(data=df_city.toarray(), columns=tf3.get_feature_names())

tf4 = TfidfVectorizer(ngram_range=(1, 1), lowercase=True)
df_locality = tf4.fit_transform(df['LOCALITY'])
df_locality = pd.DataFrame(data=df_locality.toarray(), columns=tf4.get_feature_names())

tf5 = TfidfVectorizer(ngram_range=(1, 1), lowercase=True)
df_time = tf5.fit_transform(df['TIME'])
df_time = pd.DataFrame(data=df_time.toarray(), columns=tf5.get_feature_names())

In [26]:
df = pd.concat([df, df_title, df_cuisines, df_city, df_locality, df_time], axis=1) 
df.drop(['TITLE', 'CUISINES', 'CITY', 'LOCALITY', 'TIME'], axis=1, inplace=True)

In [27]:
df= pd.get_dummies(df, columns=['CLOSED'], drop_first=True)

In [28]:
df.head()

,RATING,VOTES,COST,CITY_MEAN_RATING,LOCALITY_MEAN_RATING,bakery,bar,beverage,bhojanalya,bites,...,"CLOSED_Closed (Mon, Wed, Thu, Sun)",CLOSED_Closed (Mon-Thu),CLOSED_Closed (Mon-Tue),CLOSED_Closed (Sat),CLOSED_Closed (Sat-Sun),CLOSED_Closed (Sun),CLOSED_Closed (Tue),CLOSED_Closed (Wed),CLOSED_Closed (Wed-Sun),CLOSED_NA
0,3.6,49.0,1200.0,3.376271,3.388889,0.0,0.000000,0.0,0.0,0.000000,...,0,0,0,0,0,0,0,0,0,1
1,4.2,30.0,1500.0,3.584588,3.472222,0.0,0.808335,0.0,0.0,0.000000,...,0,0,0,0,0,0,0,0,0,1
2,3.8,221.0,800.0,3.584588,3.550000,0.0,0.000000,0.0,0.0,0.000000,...,0,0,0,0,0,0,0,0,0,1
3,4.1,24.0,800.0,3.697880,3.721622,0.0,0.000000,0.0,0.0,0.707107,...,0,0,0,0,0,0,0,0,0,1
4,3.8,165.0,300.0,3.697880,3.986420,0.0,0.000000,0.0,0.0,0.000000,...,0,0,0,0,0,0,0,0,0,1


In [29]:
df.shape

(16921, 2283)

In [30]:
train_df = df[df['COST'].isnull()!=True]
test_df = df[df['COST'].isnull()==True]x
test_df.drop('COST', axis=1, inplace=True)

In [31]:
train_df.shape, test_df.shape

((12690, 2283), (4231, 2282))

In [32]:
train_df['COST'] = np.log1p(train_df['COST'])

In [33]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 12690 entries, 0 to 12689
Columns: 2283 entries, RATING to CLOSED_NA
dtypes: float64(2262), uint8(21)
memory usage: 219.4 MB


# Modelling

In [34]:
train_df.shape, test_df.shape

((12690, 2283), (4231, 2282))

In [35]:
X = train_df.drop(labels=['COST'], axis=1)
y = train_df['COST'].values

from sklearn.model_selection import train_test_split
X_train, X_cv, y_train, y_cv = train_test_split(X, y, test_size=0.25, random_state=1)

In [36]:
X_train.shape, y_train.shape, X_cv.shape, y_cv.shape

((9517, 2282), (9517,), (3173, 2282), (3173,))

In [37]:
from math import sqrt 
from sklearn.metrics import mean_squared_log_error

In [38]:
from sklearn.ensemble import BaggingRegressor
br = BaggingRegressor(base_estimator=None, n_estimators=30, max_samples=0.9, max_features=1.0, bootstrap=True, 
                      bootstrap_features=True, oob_score=True, warm_start=False, n_jobs=1, random_state=42, verbose=1)
br.fit(X_train, y_train)
y_pred_br = br.predict(X_cv)
print('RMSLE:', sqrt(mean_squared_log_error(np.exp(y_cv), np.exp(y_pred_br))))

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   45.1s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


RMSLE: 0.358025533933587


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    2.8s finished


In [39]:
from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor(n_estimators=40, criterion='mse', max_depth=None, min_samples_split=4, min_samples_leaf=1, 
                           min_weight_fraction_leaf=0.0, max_features='auto', max_leaf_nodes=None, 
                           min_impurity_decrease=0.0, min_impurity_split=None, bootstrap=True, oob_score=False, n_jobs=1, 
                           random_state=42, verbose=1, warm_start=False)
rf.fit(X_train, y_train)
y_pred_rf = rf.predict(X_cv)
print('RMSLE:', sqrt(mean_squared_log_error(np.exp(y_cv), np.exp(y_pred_rf))))

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  40 out of  40 | elapsed:   47.3s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  40 out of  40 | elapsed:    0.0s finished


RMSLE: 0.36191172768713686


In [40]:
y_pred = y_pred_br*0.15 +  y_pred_rf*0.15
print('RMSLE:', sqrt(mean_squared_log_error(np.exp(y_cv), np.exp(y_pred))))

RMSLE: 4.244364076361807


In [41]:
#saving the model
import joblib
joblib.dump(rf,'Reastaurant food price prediction_rf.pk1')

['Reastaurant food price prediction_rf.pk1']

Prediction on test

In [42]:
Xtest = test_df

In [43]:
X_train.shape, y_train.shape, Xtest.shape

((9517, 2282), (9517,), (4231, 2282))

In [44]:
from sklearn.model_selection import KFold
from sklearn.ensemble import BaggingRegressor

err_br = []
y_pred_totbr = []

fold = KFold(n_splits=15, shuffle=True, random_state=42)

for train_index, test_index in fold.split(X):
    X_train, X_test = X.loc[train_index], X.loc[test_index]
    y_train, y_test = y[train_index], y[test_index]

    br = BaggingRegressor(base_estimator=None, n_estimators=30, max_samples=1.0, max_features=1.0, bootstrap=True,
                          bootstrap_features=True, oob_score=False, warm_start=False, n_jobs=1, random_state=42, verbose=0)
    
    br.fit(X_train, y_train)
    y_pred_br = br.predict(X_test)

    print("RMSE BR:", sqrt(mean_squared_log_error(np.exp(y_test), np.exp(y_pred_br))))

    err_br.append(sqrt(mean_squared_log_error(np.exp(y_test), np.exp(y_pred_br))))
    p = br.predict(Xtest)
    y_pred_totbr.append(p)

RMSE BR: 0.3612616106739575
RMSE BR: 0.3607758124597734
RMSE BR: 0.3500811163397139
RMSE BR: 0.35296193365175416
RMSE BR: 0.33760397605720915
RMSE BR: 0.33232279472856197
RMSE BR: 0.3639967361374431
RMSE BR: 0.3788228366234768
RMSE BR: 0.357627244338562
RMSE BR: 0.3394908170236888
RMSE BR: 0.3442295049644499
RMSE BR: 0.3569023653800222
RMSE BR: 0.3401987795249517
RMSE BR: 0.3573444559089423
RMSE BR: 0.369779731668843


In [45]:
from sklearn.model_selection import KFold
from sklearn.ensemble import RandomForestRegressor

err_rf = []
y_pred_totrf = []

fold = KFold(n_splits=15, shuffle=True, random_state=42)

for train_index, test_index in fold.split(X):
    X_train, X_test = X.loc[train_index], X.loc[test_index]
    y_train, y_test = y[train_index], y[test_index]

    rf = RandomForestRegressor(n_estimators=40, criterion='mse', max_depth=None, min_samples_split=4, min_samples_leaf=1, 
                           min_weight_fraction_leaf=0.0, max_features='auto', max_leaf_nodes=None, 
                           min_impurity_decrease=0.0, min_impurity_split=None, bootstrap=True, oob_score=False, n_jobs=1, 
                           random_state=42, verbose=0, warm_start=False)
    rf.fit(X_train, y_train)
    y_pred_rf = rf.predict(X_test)

    print("RMSE RF: ", sqrt(mean_squared_log_error(np.exp(y_test), np.exp(y_pred_rf))))

    err_rf.append(sqrt(mean_squared_log_error(np.exp(y_test), np.exp(y_pred_rf))))
    p = rf.predict(Xtest)
    y_pred_totrf.append(p)

RMSE RF:  0.3673467575976511
RMSE RF:  0.3678317420298845
RMSE RF:  0.3539778731985246
RMSE RF:  0.35402274258860855
RMSE RF:  0.34346906437843333
RMSE RF:  0.33934024433322235
RMSE RF:  0.3718529995624975
RMSE RF:  0.3829743399830808
RMSE RF:  0.3630981682439362
RMSE RF:  0.3514400386992818
RMSE RF:  0.3575681479085334
RMSE RF:  0.3630032166189251
RMSE RF:  0.3489038915227847
RMSE RF:  0.36813862078063203
RMSE RF:  0.37391291112766245


In [46]:
 np.mean(err_br,0), np.mean(err_rf,0)

(0.35355998103209, 0.3604587172382439)

In [47]:

br_final = np.exp(np.mean(y_pred_totbr,0))
rf_final = np.exp(np.mean(y_pred_totrf,0))

In [48]:

y_pred = ( br_final*0.215 + rf_final*.15) 
y_pred

array([396.15911653, 150.30199807, 234.80592951, ..., 144.32004775,
        91.73522571, 132.4229042 ])